**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [16]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Flatten, Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [17]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [18]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

This function returns what action the agent will do given the current state $s_t$ and it will depend on all the previous cumulated knowledge.

Actions in this case will be too biased by the past during the training process. In order to avoid this, it is necessary to add an exploration criteria that guides the agent to act randomly and not act based only on what it has learned. Epsilon is then essential and allows to control this exploration. This parameter should be set carrefully in order to get a good performance. if it is equal to 0, then the agent will act based on what it has learned only. In the other hand, if it is equal to 1, then the agent will act randomly in each time step.

A pure exploitation strategy can be used in the act function in the test phase when the agent is already trained and thus it needs no exploration. In this case epsilon=0 ( train=False in the act function).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [19]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [67]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=12 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

- The array position contains informamations about where the rat can go: it contains ___0___ where the rat can actually go, ___1___ in the position where the rat is and finally __-1__ in the boundaries where it can't go.

- The array board contain the rewards associated to each celle (x,y) that the rat can get if it come to this cell. Each cell of this array can be a bonus ( reward=+0.5) with a probability temperature,a malus(reward=-1) with probaility temperature or 0(without reward).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [68]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [69]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [70]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.0/17.0. Average score (-6.0)
Win/lose count 12.0/18.0. Average score (-6.0)
Win/lose count 15.0/13.0. Average score (-3.3333333333333335)
Win/lose count 12.5/11.0. Average score (-2.125)
Win/lose count 11.5/20.0. Average score (-3.4)
Win/lose count 9.0/13.0. Average score (-3.5)
Win/lose count 9.5/13.0. Average score (-3.5)
Win/lose count 5.0/13.0. Average score (-4.0625)
Win/lose count 13.0/17.0. Average score (-4.055555555555555)
Win/lose count 9.5/18.0. Average score (-4.5)
Win/lose count 11.5/16.0. Average score (-4.5)
Win/lose count 9.5/13.0. Average score (-4.416666666666667)
Final score: -4.416666666666667


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Question 5.1__

We have by definition : 

\begin{align*}
Q^{\pi}(s,a)&=E_{\pi}[\sum_{0\leq t}\gamma^tr(s_t,a_t)\vert s_0=s,a_0=a] \\
&= E_\pi[r(s,a)] +  E_{\pi}[ \gamma \sum_{0\leq t}\gamma^tr(s_{t+1},a_{t+1})\vert s_0=s,a_0=a] \\
&= r(s,a) + \sum_{(a',s')} \gamma E_{\pi}[  \sum_{0\leq t}\gamma^tr(s_{t+1},a_{t+1})\vert s_0=s,a_0=a,s_1=s',a_1=a'] p(s_1=s',a_1=a'|s_0=s,a_0=a) \\
&= r(s,a) + \sum_{(a',s')} \gamma E_{\pi}[  \sum_{0\leq t}\gamma^tr(s_{t+1},a_{t+1})\vert s_1=s',a_1=a'] p(s_1=s',a_1=a'|s_0=s,a_0=a)  \\
&= r(s,a) + \sum_{(a',s')}p(s_1=s',a_1=a'|s_0=s,a_0=a)  \gamma Q^{\pi}(s',a') \\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\ 
\end{align*}

__Question 5.2__

\begin{align*}
    Q^*(s,a)
    &= \max_{\pi}Q^\pi(s,a) \\
    &=  \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\ 
    &=  r(s,a) +  \max_{\pi} \sum_{(a',s')} p(s_1=s', a_1=a' |s_{0}=s,a_{0}=a) \gamma Q^{\pi}(s',a') \\   
    &=  r(s,a) +  \max_{\pi} \sum_{(a',s')}  p(s_1=s'|s_{0}=s,a_{0}=a) \pi(a' | s') \gamma Q^{\pi}(s',a') \\ 
    &=  r(s,a) +   \sum_{(a',s')} p(s_1=s'|s_{0}=s,a_{0}=a)   \max_{\pi} \pi(a' | s') \gamma Q^{\pi}(s',a') \\ 
    &=  r(s,a) +   \sum_{(a',s')}  p(s_1=s'|s_{0}=s,a_{0}=a)  \gamma \max_{a'} Q^{*}(s',a') \\ 
    &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] \\
\end{align*}

__Question 5.3__

The role of the loss function is to measure how far our prediction is from actual target. In this case we want to minimize the gap between our model prediction $Q(s,a,\theta)$ and the target $E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$ . Thus, a plausible objective that we have to minimize here is the loss:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*} 

Using this loss function we try to enforce our learned Q-value to verify the Bellman equation


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [71]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]
            

    def random_access(self):
        element=np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[element]

***
The pipeline we will use for training is given below:

In [72]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [73]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        S_reshaped=s.reshape([1,s.shape[0],s.shape[1],s.shape[2]])
        act= np.argmax(self.model.predict(S_reshaped)[0,:])
        return act

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            # Initializing parameters
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            S__reshaped=s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]])
            target_q[i] =self.model.predict(S__reshaped)[0]
            
            if game_over_:
                target_q[i,a_] = r_
                input_states[i] = s_
                
            else:  
                n_s_reshaped=n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]])
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_reshaped)[0])
                input_states[i] = s_
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model=Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32,activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [74]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0225 | Win/lose count 8.0/8.0 (0.0)
Epoch 001/050 | Loss 0.0130 | Win/lose count 7.0/8.0 (-1.0)
Epoch 002/050 | Loss 0.0219 | Win/lose count 5.5/4.0 (1.5)
Epoch 003/050 | Loss 0.0180 | Win/lose count 6.0/4.0 (2.0)
Epoch 004/050 | Loss 0.0107 | Win/lose count 2.0/5.0 (-3.0)
Epoch 005/050 | Loss 0.0129 | Win/lose count 1.0/6.0 (-5.0)
Epoch 006/050 | Loss 0.0488 | Win/lose count 2.5/3.0 (-0.5)
Epoch 007/050 | Loss 0.0048 | Win/lose count 5.0/3.0 (2.0)
Epoch 008/050 | Loss 0.0044 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/050 | Loss 0.0482 | Win/lose count 7.5/3.0 (4.5)
Epoch 010/050 | Loss 0.0088 | Win/lose count 5.0/4.0 (1.0)
Epoch 011/050 | Loss 0.0063 | Win/lose count 7.0/3.0 (4.0)
Epoch 012/050 | Loss 0.0056 | Win/lose count 6.0/3.0 (3.0)
Epoch 013/050 | Loss 0.0034 | Win/lose count 4.0/3.0 (1.0)
Epoch 014/050 | Loss 0.0045 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/050 | Loss 0.0035 | Win/lose count 6.5/4.0 (2.5)
Epoch 016/050 | Loss 0.0076 | Win/lose count 2.5/1.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [75]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        nb_neurons=50
        
        model = Sequential()
        model.add(Conv2D(nb_neurons,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [76]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0054 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/050 | Loss 0.0059 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/050 | Loss 0.0098 | Win/lose count 5.0/8.0 (-3.0)
Epoch 003/050 | Loss 0.0016 | Win/lose count 3.0/0 (3.0)
Epoch 004/050 | Loss 0.0033 | Win/lose count 1.0/1.0 (0.0)
Epoch 005/050 | Loss 0.0043 | Win/lose count 2.0/3.0 (-1.0)
Epoch 006/050 | Loss 0.0144 | Win/lose count 4.5/4.0 (0.5)
Epoch 007/050 | Loss 0.0014 | Win/lose count 2.5/0 (2.5)
Epoch 008/050 | Loss 0.0416 | Win/lose count 3.5/1.0 (2.5)
Epoch 009/050 | Loss 0.1109 | Win/lose count 6.0/3.0 (3.0)
Epoch 010/050 | Loss 0.0070 | Win/lose count 5.0/0 (5.0)
Epoch 011/050 | Loss 0.0543 | Win/lose count 9.0/2.0 (7.0)
Epoch 012/050 | Loss 0.0015 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/050 | Loss 0.0025 | Win/lose count 8.0/2.0 (6.0)
Epoch 014/050 | Loss 0.0033 | Win/lose count 8.0/1.0 (7.0)
Epoch 015/050 | Loss 0.0038 | Win/lose count 8.0/2.0 (6.0)
Epoch 016/050 | Loss 0.0026 | Win/lose count 4.0/3.0 (1.0)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.


I have performed several tests changing the temperature parameter. What we can see is that:

- The CNN model largely outperforms the FCC model, this result can be seen in the training as well as in test scores. We have observed also that increasing the temperature increases the final reward and that The gap between the performances of the two medels is increasing with the temperature.
- The learning process can encourage in many cases the agent to stay in a visited cell where the reward is equal to 0 instead of exploring neighbor malus cells. Besides, the rat can also get stuck in many cells situated next to the boarder, this is due to the lack of space exploration.

In [78]:
env = Environment(grid_size=size, max_time=T,temperature=0.7)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 16.0/1.0. Average score (15.0)
Win/lose count 24.5/1.0. Average score (19.25)
Win/lose count 30.5/5.0. Average score (21.333333333333332)
Win/lose count 44.0/1.0. Average score (26.75)
Win/lose count 27.0/4.0. Average score (26.0)
Win/lose count 24.5/3.0. Average score (25.25)
Win/lose count 33.0/1.0. Average score (26.214285714285715)
Win/lose count 31.5/6.0. Average score (26.125)
Win/lose count 27.0/2.0. Average score (26.0)
Win/lose count 20.5/1.0. Average score (25.35)
Win/lose count 16.0/3.0. Average score (24.227272727272727)
Win/lose count 24.0/8.0. Average score (23.541666666666668)
Final score: 23.541666666666668
Test of the FC
Win/lose count 4.5/3.0. Average score (1.5)
Win/lose count 11.5/11.0. Average score (1.0)
Win/lose count 6.0/5.0. Average score (1.0)
Win/lose count 7.0/4.0. Average score (1.5)
Win/lose count 8.0/7.0. Average score (1.4)
Win/lose count 2.5/2.0. Average score (1.25)
Win/lose count 9.5/4.0. Average score (1.857142857142857

In [79]:
HTML(display_videos('cnn_test10.mp4'))

In [80]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [ ]:
def train_explore(agent,env,epoch,prefix=''):
    pass
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        pass
    
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [ ]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***